In [ ]:
instance = 1

## human parameters
based on De Leva (1996) 

In [ ]:
import numpy as np

# composite torso parameters ----------------------------------------------------------------------------
m_head = 0.0694
m_trunk = 0.4346

m_body = m_head + m_trunk

l_head = 0.2429
l_trunk = 0.6055
L_body = l_head + l_trunk

dCOM_head = l_trunk + l_head - 0.1215
dCOM_trunk = l_trunk - 0.3100

dCOM_body = (dCOM_head*m_head + dCOM_trunk*m_trunk)/m_body

r_head = 0.303*l_head
r_trunk = 0.328*l_trunk 

I_head = m_head*r_head**2
I_trunk = m_trunk*r_trunk**2

I_head_s = I_head + m_head*np.abs(dCOM_body-dCOM_head)**2
I_trunk_s = I_trunk + m_trunk*np.abs(dCOM_body-dCOM_trunk)**2
I_body = I_head_s + I_trunk_s

# arms --------------------------------------------------------------------------------
m_upperarm = 0.0271
m_forearm = 0.0162
m_hand = 0.0061

m_arm = m_upperarm + m_forearm + m_hand

l_upperarm = 0.3318
l_forearm = 0.2513
l_hand = 0.1899
L_arm = l_upperarm + l_forearm + l_hand

dCOM_upperarm = l_upperarm - 0.1347
dCOM_forearm = l_upperarm + l_forearm - 0.1439
dCOM_hand = l_upperarm + l_forearm + l_hand - 0.1198

dCOM_arm = (dCOM_upperarm*m_upperarm + dCOM_forearm*m_forearm + dCOM_hand*m_hand)/m_arm

r_upperarm = 0.505*l_upperarm
r_forearm = 0.278*l_forearm
r_hand = 0.288*l_hand

I_upperarm = m_upperarm*r_upperarm**2
I_forearm = m_forearm*r_forearm**2
I_hand = m_hand*r_hand**2

I_upperarm_s = I_upperarm + m_upperarm*np.abs(dCOM_arm-dCOM_upperarm)**2
I_forearm_s = I_forearm + m_forearm*np.abs(dCOM_arm-dCOM_forearm)**2
I_hand_s = I_hand + m_hand*np.abs(dCOM_arm-dCOM_hand)**2
I_arm = I_upperarm_s + I_forearm_s + I_hand_s

# legs ----------------------------------------------------
m_thigh = 0.1416
m_shank = 0.04330

L_thigh = 0.4222
L_shank = 0.4340

r_thigh = 0.329*L_thigh
r_shank = 0.251*L_shank

I_thigh = m_thigh*r_thigh**2
I_shank = m_shank*r_shank**2

# parameter
mass = {'body': m_body,
       'arm': m_arm, 
       'thigh': m_thigh,
       'shank': m_shank}

length = {'body': L_body,
          'arm': L_arm,
          'thigh': L_thigh,
          'shank': L_shank}

dCOM = {'body': dCOM_body/L_body,
        'arm': dCOM_arm/L_arm,
        'thigh': 0.4095,
        'shank': 0.4459}

inertia = {'body': I_body,
           'arm': I_arm, 
           'thigh': I_thigh,
           'shank': I_shank}

In [ ]:
# DERIVE THE EOMs SYMBOLICALLY --------------------------------------------------------------------------------------------

# import libraries
import sympy as sym
import numpy as np
import pickle as pickle

sym.init_printing()
from IPython.display import display #for pretty printing

# system parameters

g = 9.81
mb,ml1,ml2,ma = sym.symbols(['m_{b}','m_{l1}','m_{l2}','m_{a}']) # mass
lb,ll1,ll2,la = sym.symbols(['l_{b}','l_{l1}','l_{l2}','l_{a}']) # length
Inb,Inl1,Inl2,Ina = sym.symbols(['In_{b}','In_{l1}','In_{l2}','In_{a}']) # mass

# generalized coordinates

sides = ['R','L']
x,y,thb = sym.symbols(['x','y','\\theta_{b}']) 
dx,dy,dthb = sym.symbols(['\dot{x}','\dot{y}','\dot{\\theta}_{b}']) 
ddx,ddy,ddthb = sym.symbols(['\ddot{x}','\ddot{y}','\ddot{\\theta}_{b}']) 

thl1  = [sym.symbols('\\theta_{l1%s}'%sides[i]) for i in range(len(sides))]
thl2  = [sym.symbols('\\theta_{l2%s}'%sides[i]) for i in range(len(sides))]
tha  = [sym.symbols('\\theta_{a%s}'%sides[i]) for i in range(len(sides))]

dthl1  = [sym.symbols('\dot{\\theta}_{l1%s}'%sides[i]) for i in range(len(sides))]
dthl2  = [sym.symbols('\dot{\\theta}_{l2%s}'%sides[i]) for i in range(len(sides))]
dtha  = [sym.symbols('\dot{\\theta}_{a%s}'%sides[i]) for i in range(len(sides))]

ddthl1  = [sym.symbols('\ddot{\\theta}_{l1%s}'%sides[i]) for i in range(len(sides))]
ddthl2  = [sym.symbols('\ddot{\\theta}_{l2%s}'%sides[i]) for i in range(len(sides))]
ddtha  = [sym.symbols('\ddot{\\theta}_{a%s}'%sides[i]) for i in range(len(sides))]

q = sym.Matrix([x]+[y]+[thb]+thl1+thl2+tha)
dq = sym.Matrix([dx]+[dy]+[dthb]+dthl1+dthl2+dtha)
ddq = sym.Matrix([ddx]+[ddy]+[ddthb]+ddthl1+ddthl2+ddtha)

# forces
taul1  = [sym.symbols('\\tau_{l1%s}'%sides[i]) for i in range(len(sides))]
taul2  = [sym.symbols('\\tau_{l2%s}'%sides[i]) for i in range(len(sides))]
taua  = [sym.symbols('\\tau_{a%s}'%sides[i]) for i in range(len(sides))]

Gx  = [sym.symbols('G_{x%s}'%sides[i]) for i in range(len(sides))]
Gy  = [sym.symbols('G_{y%s}'%sides[i]) for i in range(len(sides))]

# STEP 1: position vectors, velocities and jacobians -----------------------------------------------------------------------
r_com = dCOM['body'] # distance from hip to body COM as fraction of body length
r_sh = 0.5319/length['body'] # distance from hip to shoulder as fraction of body length

rb = sym.Matrix([[x - r_com*lb*sym.sin(thb)],
                [y + r_com*lb*sym.cos(thb)],
                [thb]])

p_shoulder = sym.Matrix([[x - r_sh*lb*sym.sin(thb)],
                         [y + r_sh*lb*sym.cos(thb)]])

rl1 = [] # link positions
rl2 = []
ra = []

p_knee = []
p_foot = []

Jb = rb.jacobian(q) # jacobians
Jl1 = [] 
Jl2 = []
Ja = []

vb = Jb*dq # velocities
vl1 = [] 
vl2 = []
va = []

v_foot = [] # foot velocities

for i in range(len(sides)):
    # legs
    rl1.append(sym.Matrix([[x + dCOM['thigh']*ll1*sym.sin(thl1[i])],
                          [y - dCOM['thigh']*ll1*sym.cos(thl1[i])],
                          [thl1[i]]]))
        
    p_knee.append(sym.Matrix([[x + ll1*sym.sin(thl1[i])],
                             [y - ll1*sym.cos(thl1[i])]]))
        
    rl2.append(sym.Matrix([[p_knee[i][0] + dCOM['shank']*ll2*sym.sin(thl2[i])],
                          [p_knee[i][1] - dCOM['shank']*ll2*sym.cos(thl2[i])],
                          [thl2[i]]]))
    
    p_foot.append(sym.Matrix([[p_knee[i][0] + ll2*sym.sin(thl2[i])],
                              [p_knee[i][1] - ll2*sym.cos(thl2[i])]]))
    
    # arms
    ra.append(sym.Matrix([[p_shoulder[0] + dCOM['arm']*la*sym.sin(tha[i])],
                          [p_shoulder[1] - dCOM['arm']*la*sym.cos(tha[i])],
                          [tha[i]]]))
    
    # Jacobians
    Jl1.append(rl1[i].jacobian(q))
    Jl2.append(rl2[i].jacobian(q))
    Ja.append(ra[i].jacobian(q))
    
    # velocities
    vl1.append(Jl1[i]*dq)
    vl2.append(Jl2[i]*dq)
    va.append(Ja[i]*dq)
 
    # contact velocities
    v_foot.append(p_foot[i].jacobian(q)*dq)
    
# STEP 2: Energy --------------------------------------------------------------------------------------------------------
# mass vectors
Mb = sym.Matrix([[mb,mb,Inb]])
Ml1 = sym.Matrix([[ml1,ml1,Inl1]])
Ml2 = sym.Matrix([[ml2,ml2,Inl2]])
Ma = sym.Matrix([[ma,ma,Ina]])

T = 0.5*Mb*sym.matrix_multiply_elementwise(vb,vb)
V = mb*g*rb[1]

for i in range(len(sides)):
    T += 0.5*Ml1*sym.matrix_multiply_elementwise(vl1[i],vl1[i]) + \
    0.5*Ml2*sym.matrix_multiply_elementwise(vl2[i],vl2[i]) + \
    0.5*Ma*sym.matrix_multiply_elementwise(va[i],va[i]) 
    
    V += ml1*g*rl1[i][1] + \
    ml2*g*rl2[i][1] + \
    ma*g*ra[i][1] 
    
# STEP 4: partials of T and V ----------------------------------------------------------------------------------------------

# term 1
Lg1 = sym.zeros(len(q),1)
for i in range(len(q)):
    dT_ddq = sym.Matrix([sym.diff(T,dq[i])]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq # then get time derivative of that partial

# term 3
Lg3 = sym.Matrix([T]).jacobian(q).transpose() # partial of T in q

# term 4
Lg4 = sym.Matrix([V]).jacobian(q).transpose() # partial of U in q

# STEP 5: generalized forces -----------------------------------------------------------------------------------------------

Q = sym.zeros(len(q),1)
for i in range(len(sides)):
    tau_hip = sym.Matrix([[0],[0],[taul1[i]]])
    dir_hip = sym.Matrix([[0],[0],[dthl1[i] - dthb]])
    Q += dir_hip.jacobian(dq).transpose()*tau_hip
    
    tau_knee = sym.Matrix([[0],[0],[taul2[i]]])
    dir_knee = sym.Matrix([[0],[0],[dthl2[i] - dthl1[i]]])
    Q += dir_knee.jacobian(dq).transpose()*tau_knee
    
    tau_shoulder = sym.Matrix([[0],[0],[taua[i]]])
    dir_shoulder = sym.Matrix([[0],[0],[dtha[i] - dthb]])
    Q += dir_shoulder.jacobian(dq).transpose()*tau_shoulder
    
    F_GRF = sym.Matrix([[Gx[i]],[Gy[i]],[0]])
    r_GRF = sym.Matrix([[p_foot[i][0]],[p_foot[i][1]],[0]]) - sym.Matrix([[rl2[i][0]],[rl2[i][1]],[0]])
    M_GRF = r_GRF.cross(F_GRF)
    Q += Jl2[i].transpose()*(F_GRF + M_GRF)

# combine into EOM
EOM = Lg1 - Lg3 + Lg4 - Q

# # LAMBDIFY ------------------------------------------------------------------------------------------------------------------    
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

func_map = {'sin':sin, 'cos':cos} 

sym_list = [mb,ml1,ml2,ma,
            lb,ll1,ll2,la,
            Inb,Inl1,Inl2,Ina,
            x,y,thb]+thl1+thl2+tha+\
            [dx,dy,dthb]+dthl1+dthl2+dtha+\
            [ddx,ddy,ddthb]+ddthl1+ddthl2+ddtha+\
            taul1+taul2+taua+\
            Gx+Gy

lambEOM = {}
DOFs = [('x','b'),
        ('y','b'),
        ('th','b'),
        ('thl1','R'),
        ('thl1','L'),
        ('thl2','R'),
        ('thl2','L'),
        ('tha','R'),
        ('tha','L')]

for dof_i, dof in enumerate(DOFs):
    lambEOM.update({dof: sym.lambdify(sym_list,EOM[dof_i],modules = [func_map])})
    
lamb_pfoot = {}
lamb_vfoot = {}
for s_i, side in enumerate(['R','L']):
    for dof_i, dof in enumerate(['x','y']):
        lamb_pfoot.update({(dof,side): sym.lambdify(sym_list,p_foot[s_i][dof_i],modules = [func_map])})
        lamb_vfoot.update({(dof,side): sym.lambdify(sym_list,v_foot[s_i][dof_i],modules = [func_map])})
        
lamb_jointy = {}
for s_i, side in enumerate(['R','L']):
    lamb_jointy.update({('shoulder',side): sym.lambdify(sym_list,p_shoulder[1],modules = [func_map])})
    lamb_jointy.update({('hip',side): sym.lambdify(sym_list,y,modules = [func_map])})
    lamb_jointy.update({('knee',side): sym.lambdify(sym_list,p_knee[s_i][1],modules = [func_map])})

In [ ]:
N = 100
P = 2
sides = ['R','L']
links = ['b','l1','l2','a']
joints = ['shoulder','hip','knee']
joints_leg = ['hip','knee']
contacts = ['R','L']
ground_constraints = ['contact','dx_exclusive','sliding_ps','sliding_ng']
signs = ['ps','ng']

def create_model():
    m = ConcreteModel()

    # SETS-----------------------------------------------------------------------------------------------------------------------

    m.N = RangeSet(N) 

    P = 2
    m.P = RangeSet(P)

    sides = ['R','L']

    links = ['b','l1','l2','a']
    m.L = Set(initialize = links)

    joints = ['shoulder','hip','knee']
    m.J = Set(initialize = [(j,s) for j in joints for s in sides])

    # generalized coordinates
    DOFs = [('x','b'),
            ('y','b'),
            ('th','b'),
            ('thl1','R'),
            ('thl1','L'),
            ('thl2','R'),
            ('thl2','L'),
            ('tha','R'),
            ('tha','L')]
    m.DOF = Set(initialize = DOFs, dimen = 2) 

    WDOFs = ['x','y'] # absolute coordinates 
    m.WDOF = Set(initialize = WDOFs) 

    signs = ['ps','ng'] # sign set for positive and negative components
    m.sgn = Set(initialize = signs)

    # PARAMETERS-----------------------------------------------------------------------------------------------------------------

    m.g = Param(initialize = 9.81)

    def get_m(n, l):
        if l == 'b':
            return mass['body']
        if l == 'l1':
            return mass['thigh']
        if l == 'l2':
            return mass['shank']
        if l == 'a':
            return mass['arm']
    m.m = Param(m.L, initialize = get_m) # mass of links

    def get_len(n, l):
        if l == 'b':
            return length['body']
        if l == 'l1':
            return length['thigh']
        if l == 'l2':
            return length['shank']
        if l == 'a':
            return length['arm']
    m.len = Param(m.L, initialize = get_len) # length of links

    def calculate_In(m, l): 
        if l == 'b':
            return inertia['body']
        if l == 'l1':
            return inertia['thigh']
        if l == 'l2':
            return inertia['shank']
        if l == 'a':
            return inertia['arm']
    m.In = Param(m.L, initialize = calculate_In) # moment of inertia

    mbody = sum(m.m[l] for l in links)
    BW = mbody*m.g.value

    # VARIABLES -----------------------------------------------------------------------------------------------------------------

    # system coordinates
    m.q = Var(m.N, m.P, m.DOF) # position
    m.dq = Var(m.N, m.P, m.DOF) # velocity
    m.ddq = Var(m.N, m.P, m.DOF) # acceleration

    m.force_a = Var(m.N, m.J) # actuator 

    for n in range(1,N+1):
        for s in sides:
            m.force_a[n,'shoulder',s].setlb(-80/73.0)
            m.force_a[n,'shoulder',s].setub(60/73.0)
            m.force_a[n,'hip',s].setlb(-270.0/73.0)
            m.force_a[n,'hip',s].setub(180/73.0)
            m.force_a[n,'knee',s].setlb(-270/73.0)
            m.force_a[n,'knee',s].setub(150/73.0)

    # bound variables
    for n in range(1,N+1):
        for p in range(1,P+1):
            m.q[n,p,('y','b')].setlb(0)
            m.q[n,p,('th','b')].setlb(-np.pi/2)
            m.q[n,p,('th','b')].setub(np.pi/2)

    # TIME AND INTEGRATION ---------------------------------------------------------------------------------------------------

    # variable timestep
    m.hm = Param(initialize = 0.025, mutable = True) # master timestep
    m.h = Var(m.N, bounds = (0.8,1.2))

    # Integration constraints 
    radau_mat = [[0.416666666666667, -0.0833333333333333],
                 [0.75, 0.25]]

    def Radau_p(m,n,p,dof1,dof2):
        dof = (dof1,dof2)
        if n == 1:
            return Constraint.Skip
        vel = m.hm*m.h[n]*sum([radau_mat[p-1][pp-1]*m.dq[n,pp,dof] for pp in range(1,3)])
        return m.q[n,p,dof] == m.q[n-1,2,dof] + vel
    m.integrate_p2 = Constraint(m.N, m.P, m.DOF, rule = Radau_p)

    def Radau_v(m,n,p,dof1,dof2):
        dof = (dof1,dof2)
        if n == 1:
            return Constraint.Skip
        acc = m.hm*m.h[n]*sum([radau_mat[p-1][pp-1]*m.ddq[n,pp,dof] for pp in range(1,3)])
        return m.dq[n,p,dof] == m.dq[n-1,2,dof] + acc
    m.integrate_v2 = Constraint(m.N, m.P, m.DOF, rule = Radau_v)

    # JOINT CONSTRAINTS ----------------------------------------------------------------------------------------------------------
    contacts = ['R','L']
    m.C = Set(initialize = contacts)
    m.GRF = Var(m.N, m.P, m.C, m.WDOF)

    def get_var_list(m,n,p):
        var_list = [m.m[l] for l in links]+\
                   [m.len[l] for l in links]+\
                   [m.In[l] for l in links]+\
                   [m.q[n,p,dof] for dof in DOFs]+\
                   [m.dq[n,p,dof] for dof in DOFs]+\
                   [m.ddq[n,p,dof] for dof in DOFs]+\
                   [m.force_a[n,'shoulder',s] for s in sides]+\
                   [m.force_a[n,j,s] + 9.81*m.force_r[n,p,j,s,'ps'] - 9.81*m.force_r[n,p,j,s,'ng'] for j in joints_leg for s in sides]+\
                   [9.81*m.GRF[n,p,s,dof] for dof in WDOFs for s in sides]
        return var_list

    joints_leg = ['hip','knee']
    m.Jleg = Set(initialize = [(j,s) for j in joints_leg for s in sides])

    m.force_r = Var(m.N, m.P, m.Jleg, m.sgn, bounds = (0,None)) # rebound force

    m.joint_penalty = Var(m.N, m.P, m.Jleg, m.sgn, bounds = (0,None))

    m.jointp = Var(m.N, m.P, m.J)
    m.jointv = Var(m.N, m.P, m.J)
    m.jointy = Var(m.N, m.P, m.J, bounds = (0,None))
    m.power = Var(m.N, m.P, m.J)

    for n in range(1,N+1):
        for p in range(1,P+1):
            for s in sides:
                m.power[n,p,'shoulder',s].setlb(-110/73.0)
                m.power[n,p,'shoulder',s].setub(110/73.0)
                m.power[n,p,'hip',s].setlb(-3000/73.0)
                m.power[n,p,'hip',s].setub(1700/73.0)
                m.power[n,p,'knee',s].setlb(-630/73.0)
                m.power[n,p,'knee',s].setub(1100/73.0)

    def def_jointp(m,n,p,j1,j2):
        j = (j1,j2)
        if n == 1 and p < P: return Constraint.Skip
        if j1 == 'shoulder':
            return m.jointp[n,p,j] == m.q[n,p,'tha',j2] - m.q[n,p,'th','b']
        if j1 == 'hip':
            return m.jointp[n,p,j] == m.q[n,p,'thl1',j2] - m.q[n,p,'th','b']
        if j1 == 'knee':
            return m.jointp[n,p,j] == m.q[n,p,'thl2',j2] - m.q[n,p,'thl1',j2]
    m.def_jointp = Constraint(m.N, m.P, m.J, rule = def_jointp)

    def def_jointv(m,n,p,j1,j2):
        j = (j1,j2)
        if n == 1: return Constraint.Skip
        if j1 == 'shoulder':
            return m.jointv[n,p,j] == m.dq[n,p,'tha',j2] - m.dq[n,p,'th','b']
        if j1 == 'hip':
            return m.jointv[n,p,j] == m.dq[n,p,'thl1',j2] - m.dq[n,p,'th','b']
        if j1 == 'knee':
            return m.jointv[n,p,j] == m.dq[n,p,'thl2',j2] - m.dq[n,p,'thl1',j2]
    m.def_jointv = Constraint(m.N, m.P, m.J, rule = def_jointv)

    def def_jointy(m,n,p,j1,j2):
        j = (j1,j2)
        var_list = get_var_list(m,n,p)
        return m.jointy[n,p,j] == lamb_jointy[j](*var_list)
    m.def_jointy = Constraint(m.N, m.P, m.J, rule = def_jointy)

    def def_power(m,n,p,j1,j2):
        j = (j1,j2)
        if n == 1: return Constraint.Skip
        return m.power[n,p,j] == m.force_a[n,j]*m.jointv[n,p,j]
    m.def_power = Constraint(m.N, m.P, m.J, rule = def_power)

    # complementarity hard stops ------------------------------------------------------------------------------------

    def set_jlim(m,j1,j2,sgn):
        if j1 == 'hip':
            if sgn == 'ps':
                return np.pi/2
            if sgn == 'ng':
                return -np.pi/10
        if j1 == 'knee':
            if sgn == 'ps':
                return 0
            if sgn == 'ng':
                return -np.pi/2
    m.jlim = Param(m.Jleg, m.sgn, initialize = set_jlim)

    m.joint_var = Var(m.N, m.P, m.Jleg, m.sgn, bounds = (0,None))
    def def_joint_var(m,n,p,j1,j2,sgn):
        j = (j1,j2)
        if sgn == 'ps':
            return m.joint_var[n,p,j,sgn] == m.jlim[j,'ps']-m.jointp[n,p,j]
        if sgn == 'ng':
            return m.joint_var[n,p,j,sgn] == m.jointp[n,p,j]-m.jlim[j,'ng']
    m.def_joint_var = Constraint(m.N, m.P, m.Jleg, m.sgn, rule = def_joint_var)
                
    def joint_complementarity(m,n,p,j1,j2,sgn):
        j = (j1,j2)
        if n == 1: return Constraint.Skip
        if n < N:
            d = m.joint_var[n,P,j,sgn]+sum([m.joint_var[n+1,pp,j,sgn] for pp in range(1,P+1)])
        else:
            d = m.joint_var[n,P,j,sgn]
        if sgn == 'ps':
            return d*m.force_r[n,p,j,'ng'] <= m.joint_penalty[n,p,j,sgn] 
        if sgn == 'ng':
            return d*m.force_r[n,p,j,'ps'] <= m.joint_penalty[n,p,j,sgn] 
    m.joint_complementarity = Constraint(m.N, m.P, m.Jleg, m.sgn, rule = joint_complementarity)

    # GROUND MODEL -----------------------------------------------------------------------------------------------------------

    # get foot position and velocity ----------------------------------------------------------------------------------------
    m.footy = Var(m.N, m.P, m.C, bounds = (0,None)) # foot position
    m.foot_dx2 = Var(m.N, m.P, m.C, m.sgn, bounds = (0,None)) 

    def def_footy(m,n,p,c):
        if n == 1 and p < P: return Constraint.Skip    
        var_list = get_var_list(m,n,p)
        return m.footy[n,p,c] == lamb_pfoot[('y',c)](*var_list)
    m.def_footy = Constraint(m.N, m.P, m.C, rule = def_footy)

    def def_footdx(m,n,p,c):
        if n == 1: return Constraint.Skip      
        var_list = get_var_list(m,n,p)
        return m.foot_dx2[n,p,c,'ps']-m.foot_dx2[n,p,c,'ng'] == lamb_vfoot[('x',c)](*var_list)
    m.def_footdx = Constraint(m.N, m.P, m.C, rule = def_footdx)

    m.alpha = Var(m.N, m.P, m.C, bounds = (0,1))
    m.mu_s = Param(initialize = 1)
    m.mu_d = Param(initialize = 0.6)
    def def_Ft(m,n,p,c):
        if n == 1: return Constraint.Skip
        delta = 8
        dx_mag = m.foot_dx2[n,p,c,'ps'] + m.foot_dx2[n,p,c,'ng']
        mu = (m.mu_s - m.mu_d)/(1 + delta*dx_mag) + m.mu_d
        Ft_mag = mu*m.GRF[n,p,c,'y']
        return m.GRF[n,p,c,'x'] == (2*m.alpha[n,p,c]-1)*Ft_mag
    m.def_Ft = Constraint(m.N, m.P, m.C, rule = def_Ft)

    # ground complementarity ------------------------------------------------------------------------------------------------
    ground_constraints = ['contact','dx_exclusive','sliding_ps','sliding_ng']
    m.ground_constraints = Set(initialize = ground_constraints) # set for indexing ground-related penalties

    m.ground_varA = Var(m.N, m.P, m.C, m.ground_constraints, bounds = (0.0, None))
    m.ground_varB = Var(m.N, m.P, m.C, m.ground_constraints, bounds = (0.0, None))
    m.ground_penalty = Var(m.N, m.P, m.C, m.ground_constraints, bounds = (0.0,None))

    def def_ground_varA(m,n,p,c,gc):
        if n == 1: return Constraint.Skip
        if gc == 'contact':
            if n < N:
                return m.ground_varA[n,p,c,gc] == m.footy[n,P,c]+sum([m.footy[n+1,pp,c] for pp in range(1,P+1)])
            else:
                return m.ground_varA[n,p,c,gc] == m.footy[n,P,c]
        if gc == 'dx_exclusive':
            return m.ground_varA[n,p,c,gc] == m.foot_dx2[n,p,c,'ps']
        if gc == 'sliding_ps':
            return m.ground_varA[n,p,c,gc] == m.foot_dx2[n,p,c,'ps']
        if gc == 'sliding_ng':
            return m.ground_varA[n,p,c,gc] == m.foot_dx2[n,p,c,'ng']
    m.def_ground_varA = Constraint(m.N, m.P, m.C, m.ground_constraints, rule = def_ground_varA)

    def def_ground_varB(m,n,p,c,gc):
        if n == 1: return Constraint.Skip
        if gc == 'contact':
            return m.ground_varB[n,p,c,gc] == m.GRF[n,p,c,'y']
        if gc == 'dx_exclusive':
            return m.ground_varB[n,p,c,gc] == m.foot_dx2[n,p,c,'ng']
        if gc == 'sliding_ps':
            return m.ground_varB[n,p,c,gc] == m.alpha[n,p,c]
        if gc == 'sliding_ng':
            return m.ground_varB[n,p,c,gc] == (1-m.alpha[n,p,c])
    m.def_ground_varB = Constraint(m.N, m.P, m.C, m.ground_constraints, rule = def_ground_varB)

    # complementarity
    def ground_complementarity(m,n,p,c,gc):
        if n == 1: return Constraint.Skip
        return  m.ground_varA[n,p,c,gc]*m.ground_varB[n,p,c,gc] <= m.ground_penalty[n,p,c,gc]
    m.ground_complementarity = Constraint(m.N, m.P, m.C, m.ground_constraints, rule = ground_complementarity)

    # EOM --------------------------------------------------------------------------------------------------------------------
    def dynamics(m,n,p,dof1,dof2):
        dof = (dof1,dof2)
        if n == 1: return Constraint.Skip    
        var_list = get_var_list(m,n,p)
        return lambEOM[dof](*var_list) == 0
    m.dynamics = Constraint(m.N, m.P, m.DOF, rule = dynamics)

    # OBJECTIVE -----------------------------------------------------------------------------------------------------------
    def MinPenalty(m):
        return sum([m.ground_penalty[n,p,c,gc] for n in range(2,N+1) for p in range(1,P+1) for c in contacts for gc in ground_constraints])+\
               sum([m.joint_penalty[n,p,j,s,sgn] for n in range(2,N+1) for p in range(1,P+1) for j in joints_leg for s in sides for sgn in signs])
    m.MinPenalty = Objective(rule = MinPenalty)

    def MinPower(m):
    #     return sum([m.power[n,p,j,s]**2 for n in range(2,N+1) for p in range(1,P+1) for j in joints for s in sides])
        return sum([m.force_a[n,j,s]**2 for n in range(2,N+1) for j in joints for s in sides])

    m.MinPower = Objective(rule = MinPower)
    m.MinPower.deactivate()

    m.max_x = Var(bounds = (0,None))
    def def_max_x(m,n,p):
        return m.q[n,p,'x','b'] <= m.max_x
    m.def_max_x = Constraint(m.N, m.P, rule = def_max_x)

    def MinDistance(m):
        return m.max_x
    m.MinDistance = Objective(rule = MinDistance)
    m.MinDistance.deactivate()
    
    return m

In [ ]:
def init_opt(): # initializes solver
    opt = SolverFactory('ipopt',executable = r'/usr/local/bin/ipopt')
    opt.options["linear_solver"] = 'ma97'
    opt.options["print_level"] = 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
    opt.options["max_iter"] = 30000 # maximum number of iterations
    opt.options["max_cpu_time"] = 1e5 # maximum cpu time in seconds
    opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
    
    opt.options["OF_acceptable_obj_change_tol"] = 1e-4
    opt.options["OF_ma86_scaling"] = 'none'
    
    return opt

In [ ]:
import pickle as pickle

def save_model(m,outname):

    outfile = open(outname,'wb')
    data = {}

    for dof in DOFs:
        data.update({('q',dof):[m.q[n,P,dof].value for n in range(1,N+1)]})
        data.update({('dq',dof):[m.dq[n,P,dof].value for n in range(1,N+1)]})
        data.update({('ddq',dof):[m.ddq[n,P,dof].value for n in range(1,N+1)]})

    for j in joints:
        for s in sides:
            data.update({('force_a',j,s):[m.force_a[n,j,s].value for n in range(1,N+1)]})
            data.update({('jointp',j,s):[m.jointp[n,P,j,s].value for n in range(1,N+1)]})
            data.update({('jointv',j,s):[m.jointv[n,P,j,s].value for n in range(1,N+1)]})
            data.update({('power',j,s):[m.power[n,P,j,s].value for n in range(1,N+1)]})
            if j != 'shoulder':
                m.force_r[1,P,j,s,'ps'].value = 0
                m.force_r[1,P,j,s,'ng'].value = 0
                data.update({('force_r',j,s):[m.force_r[n,P,j,s,'ps'].value-m.force_r[n,P,j,s,'ng'].value for n in range(1,N+1)]})
    
    for s in sides:
        data.update({('footy',j,s):[m.footy[n,P,s].value for n in range(1,N+1)]})
        m.foot_dx2[1,P,s,'ps'].value = 0
        m.foot_dx2[1,P,s,'ng'].value = 0
        data.update({('foot_dx',j,s):[m.foot_dx2[n,P,s,'ps'].value-m.foot_dx2[n,P,s,'ng'].value for n in range(1,N+1)]})
        for dof in ['x','y']:
            data.update({('GRF',s,dof):[m.GRF[n,P,s,dof].value for n in range(1,N+1)]})
    
    pickle.dump(data,outfile)
    outfile.close()

In [ ]:
def check_model(m,result):
    if result.solver.status != SolverStatus.ok:
        print('fail: solver not ok')
        return 0
    if result.solver.termination_condition != TerminationCondition.optimal:
        print('fail: did not find optimal solution')
        return 0
    max_ground_penalty = np.max([m.ground_penalty[n,p,c,gc].value for n in range(2,N+1) for p in range(1,P+1) for c in contacts for gc in ground_constraints])
    max_joint_penalty = np.max([m.joint_penalty[n,p,j,s,sgn].value for n in range(2,N+1) for p in range(1,P+1) for j in joints_leg for s in sides for sgn in signs])
    thr = 1e-4
    if max_ground_penalty > thr:
        print('fail: ground penalty violated')
        return 0
    if max_joint_penalty > thr:
        print('fail: joint penalty violated')
        return 0
    print('success')
    return 1

In [ ]:
def stop(m,poi):
    infile = open('new_sprint','rb')
    ss_in = pickle.load(infile)
    infile.close()

    #initial condition
    for dof in DOFs:
        if dof[0] == 'tha':
            m.q[1,P,dof].setlb(ss_in['q',('th','b')][poi]-np.pi/4)
            m.q[1,P,dof].setub(ss_in['q',('th','b')][poi]+np.pi/4)
            m.dq[1,P,dof].setlb(-np.pi)
            m.dq[1,P,dof].setub(np.pi)
        else:
            m.q[1,P,dof].fix(ss_in['q',dof][poi])
            m.dq[1,P,dof].fix(ss_in['dq',dof][poi])

    m.q[1,P,'x','b'].fix(0)
                    
    # final
    for n in range(N-5,N+1):
        for p in range(1,P+1):
            for dof in DOFs:
                if dof[0] == 'tha':
                    m.dq[n,p,dof].setlb(-0.05)
                    m.dq[n,p,dof].setub(0.05)
                else:
                    m.dq[n,p,dof].setlb(-0.05*np.abs(m.dq[1,P,dof].value))
                    m.dq[n,p,dof].setub(0.05*np.abs(m.dq[1,P,dof].value))
            m.dq[n,p,'x','b'].setub(0)
            for c in contacts:
                m.footy[n,p,c].fix(0)
        
    return m

In [ ]:
def scatter(m):
    xguide = np.linspace(0,np.random.uniform(3,6),N)
    for n in range(2,N+1):
        for p in range(1,P+1):
            m.q[n,p,'x','b'].value = xguide[n-1]
            m.q[n,p,'y','b'].value = m.q[1,P,'y','b'].value
            for dof in DOFs[2:]:
                m.q[n,p,dof].value = np.random.uniform(-0.1,0.1)
                
        for c in contacts:
            m.GRF[1,P,c,'y'].value = 0
            m.GRF[1,P,c,'x'].value = 0
            for n in range(2,N+1):
                coinflip = np.random.uniform(0,1)
                if coinflip >= 0.6: #change
                    if m.GRF[n-1,P,c,'y'].value == 0:
                        Gy = np.random.uniform(0,1)
                        for p in range(1,P+1):
                            m.GRF[n,p,c,'y'].value = Gy
                            m.GRF[n,p,c,'x'].value = np.random.uniform(-1,1)*Gy
                    else:
                        for p in range(1,P+1):
                            for dof in ['x','y']:
                                m.GRF[n,p,c,dof].value = 0
                else:
                    for p in range(1,P+1):
                        for dof in ['x','y']:
                            m.GRF[n,p,c,dof].value = m.GRF[n-1,P,c,dof].value
                            
    #arms
    m.q[1,P,'tha','R'].value = np.random.uniform(m.q[1,P,'th','b'].value-np.pi/4,m.q[1,P,'th','b'].value+np.pi/4)
    m.q[1,P,'tha','L'].value = np.random.uniform(m.q[1,P,'th','b'].value-np.pi/4,m.q[1,P,'th','b'].value+np.pi/4)
    dtha = np.random.uniform(-1.5*np.pi,-6*np.pi)
    for n in range(2,N+1):
        for p in range(1,P+1):
            for s in sides:
                m.dq[n,p,'tha',s].value = dtha
                m.q[n,p,'tha',s].value = m.q[n-1,P,'tha',s].value + 0.025*m.dq[n,p,'tha',s].value
    return m
                    

In [ ]:
def activate_MinDistance(m):
    m.MinPenalty.deactivate()
    m.MinDistance.activate()

    max_ground_penalty1 = np.max([m.ground_penalty[2,p,c,gc].value for p in range(1,P+1) for c in contacts for gc in ground_constraints])
    max_joint_penalty1 = np.max([m.joint_penalty[2,p,j,s,sgn].value for p in range(1,P+1) for j in joints_leg for s in sides for sgn in signs])

    max_ground_penalty1 = 1e-4
    max_joint_penalty1 = 1e-4
    
    for n in range(1,N+1):
        for p in range(1,P+1):
            for c in contacts:
                for gc in ground_constraints:
                    if n > 3:
                        m.ground_penalty[n,p,c,gc].setub(1e-4)
                    else:
                        m.ground_penalty[n,p,c,gc].setub(np.max([max_ground_penalty1,1e-4]))
            for j in joints_leg:
                for s in sides:
                    for sgn in signs:
                        if n > 3:
                            m.joint_penalty[n,p,j,s,sgn].setub(1e-4)
                        else:
                            m.joint_penalty[n,p,j,s,sgn].setub(np.max([max_joint_penalty1,1e-4]))
                        
    return m

# solve loop

In [ ]:
opt = init_opt()

for sd in range(10,101):
    if 'm' in globals():
        del m # deletes the model
        
    m = create_model()
    
    # feasible
    m = stop(m,40)
    m = scatter(m)
    
    status = 0
    print('feasible attempt: %d'%sd)
    try:
        results = opt.solve(m, tee = False)
        
        status = check_model(m,results)
    
    except:
        print('fail')
        continue
    
    if status == 1:
        save_model(m,'results_arm/arm_stop_baseline_feasible_i%ds%d'%(instance,sd))

        # optimal
        m = activate_MinDistance(m)

        status = 0
        print('optimal attempt: %d'%sd)
        try:
            results = opt.solve(m, tee = False)

            status = check_model(m,results)

        except:
            print('fail')
            continue

        if status == 1:
            save_model(m,'results_arm/arm_stop_baseline_optimal_i%ds%d'%(instance,sd))
    else:
        continue
            
        

In [ ]:
#animate it
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig, ax = plt.subplots(1,1,figsize=[10,5],dpi=100) #create axes
ax.set_aspect('equal')

xmax = np.max([m.q[n,P,('x','b')].value for n in range(1,N+1)])
ymax = np.max([m.q[n,P,('y','b')].value for n in range(1,N+1)])


def plot_robot(i): #update function for animation
    ax.clear()
    ax.set_xlim([-1,xmax+2])
    ax.set_ylim([-0.5,ymax+1])
    ax.plot([-1,xmax+1],[0,0],color='black')
    
    #plot body
    thb = m.q[i,P,('th','b')].value
    hip_x = m.q[i,P,('x','b')].value
    hip_y = m.q[i,P,('y','b')].value
    head_x = m.q[i,P,('x','b')].value - m.len['b']*np.sin(thb)
    head_y = m.q[i,P,('y','b')].value + m.len['b']*np.cos(thb)
    ax.plot([hip_x,head_x],[hip_y,head_y],color='xkcd:black')
    
    shoulder_x = m.q[i,P,('x','b')].value - r_sh*m.len['b']*np.sin(thb)
    shoulder_y = m.q[i,P,('y','b')].value + r_sh*m.len['b']*np.cos(thb) 
    
    cols = ['xkcd:bright pink','xkcd:bright blue']
    for s_i,s in enumerate(sides):
    
        thl1 = m.q[i,P,('thl1',s)].value
        knee_x = hip_x + m.len['l1']*np.sin(thl1)
        knee_y = hip_y - m.len['l1']*np.cos(thl1)
        ax.plot([hip_x,knee_x],[hip_y,knee_y],color=cols[s_i])
        
        thl2 = m.q[i,P,('thl2',s)].value
        foot_x = knee_x + m.len['l2']*np.sin(thl2)
        foot_y = knee_y - m.len['l2']*np.cos(thl2)
        ax.plot([knee_x,foot_x],[knee_y,foot_y],color=cols[s_i])
        
        tha = m.q[i,P,('tha',s)].value
        hand_x = shoulder_x + m.len['a']*np.sin(tha)
        hand_y = shoulder_y - m.len['a']*np.cos(tha)
        ax.plot([shoulder_x,hand_x],[shoulder_y,hand_y],color=cols[s_i])
    
animate = ani.FuncAnimation(fig,plot_robot,range(1,N+1),interval = 50,repeat=True)

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook
